#### Project: Feedback summarizer

In this project, we analyze the feedback of Amazon products.

#### Imports

In [1]:
from common_functions import datasets_dir, ensure_installed, ensure_llama_running
ensure_installed(['langchain', 'langchain_community'])
ensure_llama_running()

import os

from bs4 import BeautifulSoup
from IPython.display import display, Markdown
from dotenv import load_dotenv
import pandas as pd
from selenium import webdriver

from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
llm_model = os.getenv('LLM_MODEL')

Defaulting to user installation because normal site-packages is not writeable


#### Data collection

In [2]:


def scrape_amazon_reviews(link):
	
	# get product ID from link
	product_id = link.split('/')[-1].split('?')[0]
	
	# ensure no spaces in product ID
	product_id = product_id.replace(' ', '')
	
	filename = f'reviews_{product_id}.csv'
	filepath = os.path.join(datasets_dir, filename)
	# if file exists, return the reviews
	if os.path.exists(filepath):
		return pd.read_csv(filepath)
	
	options = webdriver.ChromeOptions()
	options.add_argument('headless')  # to run the browser in background

	driver = webdriver.Chrome(options=options)
	driver.get(link)
	soup = BeautifulSoup(driver.page_source, "html.parser")
	driver.quit()
	reviews = []

	for review in soup.find_all('div', {'data-hook': 'review'}):
		title = review.find('a', {'data-hook': 'review-title'}).span.text
		# rating = review.find('i', {'data-hook': 'review-star-rating'}).span.text
		body = review.find('span', {'data-hook': 'review-body'}).span.text
		feedback = title + ' ' + body
		# remove new lines
		feedback = feedback.replace('\n', ' ')

		reviews.append({
			# 'rating': rating,
			'feedback': feedback,
		})

	reviews_df = pd.DataFrame(reviews)
	reviews_df.to_csv(filepath, index=False)
	return reviews_df

amazon_in_link = 'https://www.amazon.in/dp/B00KXULGJQ?th=1'
reviews_df = scrape_amazon_reviews(amazon_in_link)
reviews_df.head()

,feedback
0,I recently purchased the TP-Link AC750 WiFi Ra...
1,I have a range issue in the second floor as th...
2,"Excellent product, does the job very well and ..."
3,I purchased a TP Link RE450 (AC1750) Wi-Fi ext...
4,The RE450 is an 802.11ac range extender with a...


In [3]:
# convert reviews into single string
reviews = ' '.join(reviews_df['feedback'].tolist())
reviews[:100]

'I recently purchased the TP-Link AC750 WiFi Range Extender, and I must say, it has made a significan'

#### Creating the model

In [4]:
llm = Ollama(model="llama2")
llm.get_name()

'Ollama'

Define the prompt

In [5]:
# prompt can also be saved to a file and used as a template
prompt = """
Amazon reviews of a product are given below. Give me insights into what should be improved in the product based on the reviews.
"""

prompt = prompt.strip().replace("\n", " ")

In [6]:
CHARACTER_LIMIT = 2000

promptTemplate = ChatPromptTemplate.from_messages([
	("system", prompt),
	("system", reviews[:CHARACTER_LIMIT]),
	("user", "{question}"),
])
chain = promptTemplate | llm | StrOutputParser()  # Parse output as string
# Different operations are chained together to form a 'pipeline'.
# The output of one operation is passed as input to the next.

result = chain.invoke({ "question": "Please provide insights based on the reviews." })

display(Markdown(f"**Insights based on the reviews:**\n\n{result}"))

**Insights based on the reviews:**

Based on the Amazon reviews provided, here are some insights into what could be improved in the TP-Link AC750 WiFi Range Extender:

1. Ethernet Port Issue: Some reviewers have reported a strange issue with the Ethernet port on the RE450 range extender. The cause of this issue is not clear, but it may be related to hardware or software glitches. TP-Link could consider improving the Ethernet port design or troubleshooting methods to address this problem.
2. Range Extension: While the AC750 range extender effectively expanded the coverage of the WiFi network for some reviewers, others experienced range issues in certain areas. To improve the product, TP-Link could consider developing a more powerful amplifier or adjusting the antenna design to provide better coverage and signal strength.
3. Setup Ease: Some reviewers appreciated the ease of setup with WPS, but a few reported difficulties in configuring the range extender. To improve the setup process, TP-Link could consider providing more detailed instructions or adding a user-friendly interface for easier configuration.
4. Speed and Performance: While some reviewers were satisfied with the speed and performance of the AC750 range extender, others experienced slower speeds on the extension. To address this issue, TP-Link could focus on improving the hardware or software to deliver faster and more reliable connections.
5. Durability: A few reviewers mentioned that the device's antennas were prone to bending or breaking when manipulated. To improve the durability of the product, TP-Link could consider using more robust materials or designing the antennas with a more flexible structure.
6. Expandability: Some reviewers expressed interest in expanding their WiFi network beyond the range extender's capabilities. To address this issue, TP-Link could consider developing a more advanced range extender that can integrate with other devices and provide more comprehensive coverage.
7. Compatibility: A few reviewers reported compatibility issues with their existing router or device. To improve compatibility, TP-Link could ensure that the AC750 range extender is compatible with a wider range of routers and devices, or provide more detailed instructions for configuring the product with different hardware.
8. Customer Support: Some reviewers appreciated the customer support provided by TP-Link, while others experienced difficulties in getting help. To improve customer satisfaction, TP-Link could consider providing more comprehensive documentation, improving their response time to customer inquiries, or offering additional resources for troubleshooting and maintenance.

By addressing these areas of improvement, TP-Link can enhance the performance, durability, and overall value of the AC750 WiFi Range Extender, leading to increased customer satisfaction and loyalty.